![](MSFTLogo.png)
# SQL Server Machine Learning Services
## 05 - Run Batch and Single-score Predictions using T-SQL

In this scenario, *operationalizatio*n means deploying the model to production for scoring. The integration with SQL Server makes this safe, fast and easy, because you can embed Python code in a stored procedure. To get predictions from the model based on new inputs, just call the stored procedure from an application and pass the new data. Developers simply use T-SQL calls in their code to send parameters and recieve results, either by running the predictions across data stored in a table (*batch scoring*) or sending data for a prediction and receiving an immediate result (*scoring row by row*). All the Python code needed for scoring is provided as part of the stored procedures.

### Batch scoring
The first two stored procedures illustrate the basic syntax for wrapping a Python prediction call in a stored procedure. Both stored procedures require a table of data as inputs. The name of the exact model to use is provided as input parameter to the stored procedure. The stored procedure loads the serialized model from the database table *nyc_taxi_models* table, using the **SELECT** statement in the stored procedure.If you used a version column, you can specify that with a **WHERE** clause. The serialized model is stored in the Python variable *mod* for further processing using Python.

The new cases that need to be scored are obtained from the Transact-SQL query specified in *@input_data_1*. As the query data is read, the rows are saved in the default data frame, *InputDataSet*.

Both stored procedure use functions from *sklearn* to calculate an accuracy metric, *AUC* (area under curve). Accuracy metrics such as AUC can only be generated if you also provide the target label (the *tipped* column). Predictions do not need the target label (variable *y*), but the accuracy metric calculation does.

Therefore, if you don't have target labels for the data to be scored, you can modify the stored procedure to remove the AUC calculations, and return only the tip probabilities from the features (variable X in the stored procedure).

#### PredictTipSciKitPy
Run the following T-SQL statements to create both stored procedures. This stored procedure requires a model based on the *scikit-learn* package, because it uses functions specific to that package - The data frame containing inputs is passed to the predict_proba function of the logistic regression model, mod. The predict_proba function (probArray = mod.predict_proba(X)) returns a float that represents the probability that a tip (of any amount) will be given:


In [1]:
DROP PROCEDURE IF EXISTS PredictTipSciKitPy;
GO

CREATE PROCEDURE [dbo].[PredictTipSciKitPy] (@model varchar(50), @inquery nvarchar(max))
AS
BEGIN
DECLARE @lmodel2 varbinary(max) = (select model from nyc_taxi_models where name = @model);
EXEC sp_execute_external_script
  @language = N'Python',
  @script = N'
import pickle;
import numpy;
from sklearn import metrics

mod = pickle.loads(lmodel2)
X = InputDataSet[["passenger_count", "trip_distance", "trip_time_in_secs", "direct_distance"]]
y = numpy.ravel(InputDataSet[["tipped"]])

probArray = mod.predict_proba(X)
probList = []
for i in range(len(probArray)):
  probList.append((probArray[i])[1])

probArray = numpy.asarray(probList)
fpr, tpr, thresholds = metrics.roc_curve(y, probArray)
aucResult = metrics.auc(fpr, tpr)
print ("AUC on testing data is: " + str(aucResult))

OutputDataSet = pandas.DataFrame(data = probList, columns = ["predictions"])
',	
  @input_data_1 = @inquery,
  @input_data_1_name = N'InputDataSet',
  @params = N'@lmodel2 varbinary(max)',
  @lmodel2 = @lmodel2
WITH RESULT SETS ((Score float));
END
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.011

#### PredictTipRxPy
This stored procedure uses the same inputs and creates the same type of scores as the previous stored procedure, but uses functions from the *revoscalepy* package provided with SQL Server machine learning:

In [2]:
DROP PROCEDURE IF EXISTS PredictTipRxPy;
GO

CREATE PROCEDURE [dbo].[PredictTipRxPy] (@model varchar(50), @inquery nvarchar(max))
AS
BEGIN
DECLARE @lmodel2 varbinary(max) = (select model from nyc_taxi_models where name = @model);
EXEC sp_execute_external_script 
  @language = N'Python',
  @script = N'
import pickle;
import numpy;
from sklearn import metrics
from revoscalepy.functions.RxPredict import rx_predict;

mod = pickle.loads(lmodel2)
X = InputDataSet[["passenger_count", "trip_distance", "trip_time_in_secs", "direct_distance"]]
y = numpy.ravel(InputDataSet[["tipped"]])

probArray = rx_predict(mod, X)
probList = probArray["tipped_Pred"].values 

probArray = numpy.asarray(probList)
fpr, tpr, thresholds = metrics.roc_curve(y, probArray)
aucResult = metrics.auc(fpr, tpr)
print ("AUC on testing data is: " + str(aucResult))

OutputDataSet = pandas.DataFrame(data = probList, columns = ["predictions"])
',	
  @input_data_1 = @inquery,
  @input_data_1_name = N'InputDataSet',
  @params = N'@lmodel2 varbinary(max)',
  @lmodel2 = @lmodel2
WITH RESULT SETS ((Score float));
END
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.009

## Run batch scoring using a SELECT query
The stored procedures *PredictTipSciKitPy* and *PredictTipRxPy* require two input parameters:

- The query that retrieves the data for scoring
- The name of a trained model

By passing those arguments to the stored procedure, you can select a particular model or change the data used for scoring. To use the *scikit-learn* model for scoring, call the stored procedure *PredictTipSciKitPy*, passing the model name and query string as inputs:

In [3]:
DECLARE @query_string nvarchar(max) -- Specify input query
  SET @query_string='
  select tipped, fare_amount, passenger_count, trip_time_in_secs, trip_distance,
  dbo.fnCalculateDistance(pickup_latitude, pickup_longitude,  dropoff_latitude, dropoff_longitude) as direct_distance
  from nyctaxi_sample_testing'
EXEC [dbo].[PredictTipSciKitPy] 'SciKit_model', @query_string;
GO

Displaying Top 5000 rows.

STDOUT message(s) from external script: 
AUC on testing data is: 0.5652095340855293

(660029 rows affected)

Total execution time: 00:00:18.430

Score
0.508259735691279
0.508265198866627
0.508263761864075
0.508268255479534
0.508261512853405
0.50826113645573
0.508260815676444
0.508263334539785
0.50825941844363
0.508262109906045


The stored procedure returns predicted probabilities for each trip that was passed in as part of the input query. 

> Note: If you are using SSMS (SQL Server Management Studio) for running queries, the probabilities will appear as a table in the Results pane. The Messages pane outputs the accuracy metric (AUC or area under curve) with a value of around 0.56.

To use the *revoscalepy* model for scoring, call the stored procedure *PredictTipRxPy*, passing the model name and query string as inputs:

In [4]:
DECLARE @query_string nvarchar(max) -- Specify input query
  SET @query_string='
  select tipped, fare_amount, passenger_count, trip_time_in_secs, trip_distance,
  dbo.fnCalculateDistance(pickup_latitude, pickup_longitude,  dropoff_latitude, dropoff_longitude) as direct_distance
  from nyctaxi_sample_testing'
EXEC [dbo].[PredictTipRxPy] 'revoscalepy_model', @query_string;
GO

Displaying Top 5000 rows.

STDOUT message(s) from external script: 
Rows Read: 660029, Total Rows Processed: 660029, Total Chunk Time: 0.038 seconds 
AUC on testing data is: 0.5650201916103378

(660029 rows affected)

Total execution time: 00:00:11.895

Score
0.508539321169844
0.508544730279099
0.508543254708534
0.508547740823517
0.508541109968955
0.508540714804613
0.508540416764577
0.508542904457484
0.508539051233412
0.508541680472044


## Single-row scoring
Sometimes, instead of batch scoring, you might want to pass in a single case, getting values from an application, and returning a single result based on those values. For example, you could set up an Excel worksheet, web application, or report to call the stored procedure and pass to it inputs typed or selected by users. In this section, you'll learn how to create single predictions by calling two stored procedures:

- *PredictTipSingleModeSciKitPy* is designed for single-row scoring using the *scikit-learn* model
- *PredictTipSingleModeRxPy* is designed for single-row scoring using the *revoscalepy* model

> Note: If you haven't trained a model yet, return to Step 5

Both models take as input a series of single values, such as *passenger count*, *trip distance*, and so forth. A table-valued function, *fnEngineerFeatures*, is used to convert latitude and longitude values from the inputs to a new feature, *direct distance*. Both stored procedures create a score based on the Python model.

> Note - It is important that you provide all the input features required by the Python model when you call the stored procedure from an external application. To avoid errors, you might need to cast or convert the input data to a Python data type, in addition to validating data type and data length.

Run the following code creates the *PredictTipSingleModeSciKitPy* stored procedure: 

In [5]:
DROP PROCEDURE IF EXISTS PredictTipSingleModeSciKitPy;
GO

CREATE PROCEDURE [dbo].[PredictTipSingleModeSciKitPy] (@model varchar(50), @passenger_count int = 0,
  @trip_distance float = 0,
  @trip_time_in_secs int = 0,
  @pickup_latitude float = 0,
  @pickup_longitude float = 0,
  @dropoff_latitude float = 0,
  @dropoff_longitude float = 0)
AS
BEGIN
  DECLARE @inquery nvarchar(max) = N'
  SELECT * FROM [dbo].[fnEngineerFeatures]( 
    @passenger_count,
    @trip_distance,
    @trip_time_in_secs,
    @pickup_latitude,
    @pickup_longitude,
    @dropoff_latitude,
    @dropoff_longitude)
    '
DECLARE @lmodel2 varbinary(max) = (select model from nyc_taxi_models where name = @model);
EXEC sp_execute_external_script 
  @language = N'Python',
  @script = N'
import pickle;
import numpy;

# Load model and unserialize
mod = pickle.loads(model)

# Get features for scoring from input data
X = InputDataSet[["passenger_count", "trip_distance", "trip_time_in_secs", "direct_distance"]]

# Score data to get tip prediction probability as a list (of float)
probList = []
probList.append((mod.predict_proba(X)[0])[1])

# Create output data frame
OutputDataSet = pandas.DataFrame(data = probList, columns = ["predictions"])
',
  @input_data_1 = @inquery,
  @params = N'@model varbinary(max),@passenger_count int,@trip_distance float,
    @trip_time_in_secs int ,
    @pickup_latitude float ,
    @pickup_longitude float ,
    @dropoff_latitude float ,
    @dropoff_longitude float',
    @model = @lmodel2,
    @passenger_count =@passenger_count ,
    @trip_distance=@trip_distance,
    @trip_time_in_secs=@trip_time_in_secs,
    @pickup_latitude=@pickup_latitude,
    @pickup_longitude=@pickup_longitude,
    @dropoff_latitude=@dropoff_latitude,
    @dropoff_longitude=@dropoff_longitude
WITH RESULT SETS ((Score float));
END
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.009

### PredictTipSingleModeRxPy
The following stored procedure performs scoring using the *revoscalepy* model:

In [6]:
DROP PROCEDURE IF EXISTS PredictTipSingleModeRxPy;
GO

CREATE PROCEDURE [dbo].[PredictTipSingleModeRxPy] (@model varchar(50), @passenger_count int = 0,
  @trip_distance float = 0,
  @trip_time_in_secs int = 0,
  @pickup_latitude float = 0,
  @pickup_longitude float = 0,
  @dropoff_latitude float = 0,
  @dropoff_longitude float = 0)
AS
BEGIN
DECLARE @inquery nvarchar(max) = N'
  SELECT * FROM [dbo].[fnEngineerFeatures]( 
    @passenger_count,
    @trip_distance,
    @trip_time_in_secs,
    @pickup_latitude,
    @pickup_longitude,
    @dropoff_latitude,
    @dropoff_longitude)
  '
DECLARE @lmodel2 varbinary(max) = (select model from nyc_taxi_models where name = @model);
EXEC sp_execute_external_script 
  @language = N'Python',
  @script = N'
import pickle;
import numpy;
from revoscalepy.functions.RxPredict import rx_predict;

# Load model and unserialize
mod = pickle.loads(model)

# Get features for scoring from input data
X = InputDataSet[["passenger_count", "trip_distance", "trip_time_in_secs", "direct_distance"]]

# Score data to get tip prediction probability as a list (of float)

probArray = rx_predict(mod, X)

probList = []
probList = probArray["tipped_Pred"].values

# Create output data frame
OutputDataSet = pandas.DataFrame(data = probList, columns = ["predictions"])
',
  @input_data_1 = @inquery,
  @params = N'@model varbinary(max),@passenger_count int,@trip_distance float,
    @trip_time_in_secs int ,
    @pickup_latitude float ,
    @pickup_longitude float ,
    @dropoff_latitude float ,
    @dropoff_longitude float',
    @model = @lmodel2,
    @passenger_count =@passenger_count ,
    @trip_distance=@trip_distance,
    @trip_time_in_secs=@trip_time_in_secs,
    @pickup_latitude=@pickup_latitude,
    @pickup_longitude=@pickup_longitude,
    @dropoff_latitude=@dropoff_latitude,
    @dropoff_longitude=@dropoff_longitude
WITH RESULT SETS ((Score float));
END
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.016

## Generate scores from models
After the stored procedures have been created, it is easy to generate a score based on either model. Just open a new Query window, and type or paste parameters for each of the feature columns. The seven required values are for these feature columns, in order:

- passenger_count
- trip_distance vtrip_time_in_secs
- pickup_latitude
- pickup_longitude
- dropoff_latitude
- dropoff_longitude

To generate a prediction by using the *revoscalepy* model, run the first cell, for the *scikit-learn* model, run the second cell:

In [7]:
-- Revo model:
EXEC [dbo].[PredictTipSingleModeRxPy] 'revoscalepy_model', 1, 2.5, 631, 40.763958,-73.973373, 40.782139,-73.977303

STDOUT message(s) from external script: 
Rows Read: 1, Total Rows Processed: 1, Total Chunk Time: Less than .001 seconds

(1 row affected)

Total execution time: 00:00:06.505

Score
0.526496109385618


In [8]:
-- SciKit Learn model:
EXEC [dbo].[PredictTipSingleModeSciKitPy] 'SciKit_model', 1, 2.5, 631, 40.763958,-73.973373, 40.782139,-73.977303

(1 row affected)

Total execution time: 00:00:01.927

Score
0.526213020361092


# Conclusion

You've now seen how to create a model, train it, save it, and score data using the models using Python in SQL Server. You can also use R in SQL Server to perform Machine Learning and AI workloads. For more information on that, check this reference: https://docs.microsoft.com/en-us/sql/machine-learning/tutorials/r-predictive-model-introduction?view=sql-server-ver15 